In [1]:
!python3 -m pip install --upgrade pip

In [2]:
!pip3 install arxiv==2.1.0
!pip3 install python-dotenv tiktoken
# !pip install openai==0.27.8
# !pip install openai==1.2.3
!pip install openai==1.3.4
!pip install -U duckduckgo-search==4.4

In [3]:
import os
import json
import datetime as dt
import yaml
import warnings


import arxiv
import openai
from openai import OpenAI
from dotenv import load_dotenv

from duckduckgo_search import DDGS, AsyncDDGS
import asyncio

# すべての警告を無視する
warnings.filterwarnings('ignore')

In [4]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [5]:
load_dotenv()

True

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

In [8]:
# 各ノードの名称を定義
SPLIT_SEAECH_DECISION = "split_search_decision"
# 質問
question = """最近の世界経済動向を分析し、これから市場で注目すべき3つの産業を予測してください。それぞれの産業が注目される理由も含めて説明してください。
また、将来発展するような市場についても教えてください。"""
search_query = "世界経済動向, 市場, 注目, 産業, 予測, 理由, 発展, 市場の将来"
# search_query = "世界経済動向,  市場,  注目,  産業,  予測,  理由,  発展, 2024-02-21から2025-01-01までの 市場の"

手順 1: キーワードの意味を理解する
まず、質問から重要なキーワードやフレーズを把握し、それぞれがどのような情報を求めているのかを理解する必要があります。

手順 2: 関連性のあるキーワードをグループ化する
抽出したキーワードやフレーズを、意味的に関連するもの同士でグループ化します。このプロセスにより、個別の検索クエリに分割する際の基準が得られます。

手順 3: 検索クエリを形成する
各グループから具体的な検索クエリを形成します。この際、検索エンジンが理解しやすいように、必要に応じてキーワードを組み合わせたり、検索クエリを具体化したりします。

In [9]:
# エージェントに追加するシステムプロンプト作成関数
def create_agent_system(
        system_prompt: list,  # システムからエージェントへの初期プロンプト
        team_members: str,   # メンバーの役割
):
    # システムプロンプトに自律的な働きに関する指示を追加
    system_prompt.append({"role" : "system", "content" : "Work autonomously according to your specialty, using the tools available to you."})
    system_prompt.append({"role" : "system", "content" : " Do not ask for clarification."})
    system_prompt.append({"role" : "system", "content" : " Your other team members (and other teams) will collaborate with you with their own specialties."})
    system_prompt.append({"role" : "system", "content" : f" You are chosen for a reason! You are one of the following team members: {team_members}."})
    """
    あなたの専門分野に従って自律的に働いてください。使用可能なツールを使ってください
    確認のために質問をしないでください
    あなたの他のチームメンバーや他のチームも、それぞれの専門分野であなたと協力します
    あなたが選ばれたのには理由があります！あなたは以下のチームメンバーの一人です: {team_members}
    """
    # エージェントを実行するsystem_promptを返す
    return system_prompt

In [10]:
def split_search_query(model_name, question, search_query):
    # 検索判断エージェントを呼び出し、結果を取得
    prompt = [{'role': 'system', 'content': "Based on the following 'question', please group the 'search_query' based on relevance and format it into a list that can be used in Python."}]
    
    split_s_dec_prompt = create_agent_system(prompt, SPLIT_SEAECH_DECISION)
    split_s_dec_prompt.append({"role": "system", "content": 'Please generate a JSON from the following input text. Use "split_search_query_result" as the schema, and "the result of grouping the search_query based on relevance into a list format that can be used in Python" as the key. Generate it in the format {"split_search_query_result": the result of grouping the search_query based on relevance into a list format that can be used in Python}.'})
    
    split_s_dec_prompt.append({"role": "user", "content": 'Generate a JSON from the following input text. Use "split_search_query_result" as the schema, and use the judgment result as the key, to create it in the format {"split_search_query_result": the result of grouping the search_query based on relevance into a list format that can be used in Python}.'})
    split_s_dec_prompt.append({"role": "user", "content": f"Input text: {question}"})
    split_s_dec_prompt.append({"role": "user", "content": f"Search queries extracted from the input text: {search_query}"})
    """
    システム
    あなたは、以下の question に基づいて、 search_query を関連性に基づいてグループ化して python で使用できるリストの形にしてください。
    
    あなたの専門分野に従って自律的に働いてください。使用可能なツールを使ってください
    確認のために質問をしないでください
    あなたの他のチームメンバーや他のチームも、それぞれの専門分野であなたと協力します
    あなたが選ばれたのには理由があります！あなたは以下のチームメンバーの一人です: {team_members}
    
    以下の入力されたテキストからJSONを生成してください。スキーマとして「split_search_query_result」、キーとして「search_query を関連性に基づいてグループ化して python で使用できるリストの形した結果」を使用し、{"split_search_query_result": search_query を関連性に基づいてグループ化して python で使用できるリストの形した結果}の形式で生成してください。
    user
    以下の入力されたテキストからJSONを生成する。スキーマとして "split_search_query_result"を使用し、キーとして判断結果を使用して、{"split_search_query_result": search_query を関連性に基づいてグループ化して python で使用できるリストの形した結果}というフォーマットで生成します。
    
    入力されたテキスト: {question}
    入力されたテキストから抽出した検索クエリ: search_query
    """
    
    # Research用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=split_s_dec_prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    split_search_query_str = response.choices[0].message.content
    print(split_search_query_str)
    
    # JSON形式の文字列を辞書に変換
    split_search_query = json.loads(split_search_query_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "output": split_search_query["split_search_query_result"],
    }


In [11]:
# 検索クエリの分割を実行
split_queries = split_search_query(MODEL_NAME, question, search_query)

{
    "split_search_query_result": {
        "Relevance Group 1": ["世界経済動向", "市場", "市場の将来"],
        "Relevance Group 2": ["産業", "予測", "理由", "発展", "注目"]
    }
}


In [12]:
# split_search_query_result 内の全てのキーと値に順番にアクセス
for key, values in split_queries["output"].items():
    print(f"{key}: {values}")

Relevance Group 1: ['世界経済動向', '市場', '市場の将来']
Relevance Group 2: ['産業', '予測', '理由', '発展', '注目']
